In [2]:
import torch.nn as nn
import torch
import pdb

In [16]:
class DeepFM(nn.Module):
    def __init__(self,features_size,embedding_size):
        super(DeepFM,self).__init__()
        
        self.features_size = features_size
        self.field_size = len(features_size)
        self.embedding_size = embedding_size
        
        ## FM first order embedding
        self.first_order_embeddings = nn.ModuleList([nn.Embedding(feature_size,1) for feature_size in features_size])
        ## FM second order embedding
        self.second_order_embeddings = nn.ModuleList([nn.Embedding(feature_size,self.embedding_size) for feature_size in features_size])
        ## Deep part
        self.dense_dim = self.field_size * self.embedding_size
        self.fc_dim = 500
        
        
        self.fc_layer1 = nn.Sequential(
                nn.Linear(self.dense_dim,self.fc_dim),
                nn.BatchNorm1d(self.fc_dim),
                nn.LeakyReLU(0.2, inplace=True)     
                )
 
        self.fc_layer2 = nn.Sequential(
                nn.Linear(self.fc_dim,self.fc_dim),
                nn.BatchNorm1d(self.fc_dim), 
                nn.LeakyReLU(0.2, inplace=True)     
                )
        
    
    def forward(self,xi,vi):
        ## input shape: bacth_size * field_size
        
        first_order_embeddings = self.first_order_embeddings
        second_order_embeddings = self.second_order_embeddings
        field_size = self.field_size
        ## FM first order part
#         pdb.set_trace()
            
        first_order_output_tmp1 = [ (first_order_embeddings[i](xi[:,i]).t()*vi[:,i]).t() for i in range(field_size)]
        first_order_output_tmp2 = torch.cat(first_order_output_tmp1,axis=1)
        first_order_output = torch.sum(first_order_output_tmp2,axis=1)
        
        ## FM second order part
        second_order_output_tmp1 = [(second_order_embeddings[i](xi[:,i]).t()*vi[:,i]).t() for i in range(field_size) ]
        ## 计算第一个二阶项, 固定 field不变
        sum_sec_order_output  = sum(second_order_output_tmp1)
        squared_sec_order_output = sum_sec_order_output*sum_sec_order_output
        sum_squared_sec_order_output = torch.sum(squared_sec_order_output,axis=1)/2
        ## 计算第二个二阶项
        squared_sec_list = [item*item for item in second_order_output_tmp1]
        sum_squared_sec = sum(squared_sec_list)
        sum_sum_squared_sec = torch.sum(sum_squared_sec,axis=1)/2
        ## finally
        second_order_output = sum_squared_sec_order_output-sum_sum_squared_sec
        
        ## Deep part
        dense_input = torch.cat(second_order_output_tmp1,axis=1)
        fc1_output = self.fc_layer1(dense_input)
        fc2_output = self.fc_layer2(fc1_output)
        
        ## sum all together
        sum_all = first_order_output+second_order_output+ torch.sum(fc2_output,axis=1)
        return sum_all

In [17]:
## dataloader
import pickle
with open('./processed/deepfm_training.pickle','rb') as fh: 
    total_data = pickle.load(fh)

In [18]:
import numpy as np
xi =np.array(total_data['xi'],dtype=int)
vi = np.array(total_data['vi'],dtype=int)
label = np.array(total_data['label'],dtype=int)
## convert numpy.ndarray to torch tensor object
xi = torch.tensor(xi,dtype=torch.int)
vi = torch.tensor(vi,dtype=torch.int)
label=torch.tensor(label,dtype=torch.long)


In [19]:
from torch.utils.data import TensorDataset,DataLoader

In [20]:
class FM_Dataset(TensorDataset):
    def __init__(self,xi,vi,y=None):
        super(FM_Dataset,self).__init__()
        self.xi=xi
        self.vi=vi
        self.y=y
    def __len__(self):
        return self.xi.shape[0]
    def __getitem__(self,index):
        if self.y is not None: 
            sample = {'xi':self.xi[index],'vi':self.vi[index],'y':self.y[index]}
        else:
            sample = {'xi':self.xi[index],'vi':self.vi[index]}
        return sample
        

In [21]:
dataset = FM_Dataset(xi,vi,label)

In [22]:
batch_size=100
dataloader = DataLoader(dataset,shuffle=True,batch_size=batch_size)

In [23]:
##先看数据的feature_size 是多少？
## 这里只是针对这小部分数据取的feature_size
features_size=[]
for col in range(7):
    features_size.append(int(max(xi[:,col]))+1)
    

In [24]:
int(features_size[0])

1048407

In [27]:
# ## 定义损失函数
# loss = nn.CrossEntropyLoss()

features_size=features_size
embedding_size=100
# ## 定义模型
fm_model=DeepFM(features_size,embedding_size)
# 试一下forward
for data in dataloader:
    xi = torch.tensor(data['xi'],dtype=torch.long)
    vi = torch.tensor(data['vi'],dtype=torch.float32)
    print(xi.shape)
    label = torch.tensor(data['y'],dtype=torch.long)
    print(vi.shape)
    output=fm_model(xi,vi)
    
    
    break


C:\ProgramData\Anaconda3\envs\pytorch2\lib\site-packages\ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\envs\pytorch2\lib\site-packages\ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()


torch.Size([100, 7])


C:\ProgramData\Anaconda3\envs\pytorch2\lib\site-packages\ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]


torch.Size([100, 7])


In [3]:
torch.__version__

'1.2.0'